### Linear regression with NNs

In [ ]:
# Generating data 


# Dataset: {x,y}, 1D (dimesnion=1) x values with their y-values. N = 1000

# Layers of NN 2

N =20
mu_x = 5
sigma_x = 2

X = np.sort(np.random.normal(mu_x, sigma_x,N))
m_true = 2 
c_true = 3

y_with_noise = np.zeros((N),)

mu = 0
sigma = 2

for i in range(N):
    y_with_noise[i] = m_true*X[i] + c_true + np.random.normal(mu, sigma)

plt.plot(X,m_true*X+c_true)
plt.scatter(X,y_with_noise, s = 3, color = 'red')


y_with_noise = y_with_noise[:, np.newaxis]
# y` = Ax is the system. 

Matrix_A = np.vstack([X,np.ones(N)]).T

LSq_sol = np.dot(np.matmul(np.linalg.inv(np.matmul(Matrix_A.T,Matrix_A)),Matrix_A.T),y_with_noise)

m_lsq, c_lsq = LSq_sol

plt.plot(X,m_lsq*X+c_lsq, linestyle= 'dashed', color='black')

LSq_sol

In [ ]:
# Neural network training.

# Initializing
units_per_layer = 5
W_1 = np.random.normal(0, 1, size =[units_per_layer])
W_2 = np.random.normal(0, 1, size =[units_per_layer])

def sigmoid(x):
    return 1/(1+np.exp(-x))

def Relu(x):
    if type(x) == float or type(x)== int:
        if x>0:
            return x
        else:
            return 0

    else:
        for i in range(len(x)):
            if x[i]>0:
                x[i] = x[i]
                return x
            else:
                x[i] = 0
                return x

def tanh(x):
    return np.tanh(x)

def der_Relu(x):
    if type(x) == np.float64 or type(x)== int:
        if x>0:
            return 1
        else:
            return 0

    else:
        for i in range(len(x)):
            if x[i]>0:
                x[i] = 1
            else:
                x[i] = 0
        return x
    

def der_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

def tanh_prime(x):
    return 1-np.tanh(x)**2

# Forward prop

# Training

DelE_delW1 = np.random.normal(0,1,size=units_per_layer)
DelE_delW2 = np.random.normal(0,1,size=units_per_layer)
diff_y = np.zeros((N))

training_steps = 20
Error = np.zeros((training_steps))

def mse(x,y):
    return (x-y)**2


for t  in range(training_steps):
    for j in range(units_per_layer):
        for n in range(N):
            Pred_first_layer = tanh(X[n]*W_1)
            pred_second_layer = np.matmul(W_2,Pred_first_layer)
            diff_y[n] = abs(pred_second_layer-y_with_noise[n])
            DelE_delW2[j] += diff_y[n]*Pred_first_layer[j]
            DelE_delW1[j] += X[n]*tanh_prime(W_1[j]*X[n])*diff_y[n]*W_2[j]


        W_1[j] = W_1[j] - (10**-7)*DelE_delW1[j]
        W_2[j] = W_2[j] - (10**-7)*DelE_delW2[j]
        En = 0
        for n in range(N):
            Pred_first_layer = sigmoid(X[n]*W_1)
            Pred_first_layer = Pred_first_layer[:,np.newaxis]
            pred_second_layer = np.matmul(W_2,Pred_first_layer)
            En += (pred_second_layer-y_with_noise[n])**2/(2*len(y_with_noise))
    Error[t] = En/N

    
Error
## Error calculation



# def prediction_NN(X,n,W_1,W_2):
    # Pred_first_layer = sigmoid(X[n]*W_1)
    # return np.matmul(W_2,Pred_first_layer)


Pred_first_layer = sigmoid(X[0]*W_1)
np.matmul(W_2,Pred_first_layer)

Error

### Hardcoded implementation of ANN

In [ ]:
import numpy as np
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError


class FClayer(Layer):
    # input_size = no. of input neurons
    # output_size = no. of ouput neurons

    # Input is a row vector of dimension n (shape = (n,1)) and so are the biases such that Y = W*X.T + B
    def __init__(self, input_size, output_size):
        self.weights = np.random.normal(0,1,size= [input_size,output_size]) 
        self.biases = np.random.normal(0,1,size=[1,output_size]) 

    def forward_prop(self,input):
        self.input = input
        self.output = self.input @ self.weights  + self.biases
        return self.output

    def back_prop(self, output_error,learning_rate):
        dEdW =  self.input.T @ output_error 
        dEdB = output_error
        input_error = output_error @ self.weights.T

        # update params
        self.weights -= dEdW*learning_rate
        self.biases -= dEdB*learning_rate

        return input_error
    
class Activation_layer(Layer):

    def __init__(self, activation,activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_prop(self,input):
        self.input = input
        self.output = self.activation(self.input)
        return self.output

    def back_prop(self,output_error,learning_rate):
        input_error = np.multiply(output_error,self.activation_prime(self.input))
        return input_error

# Activation function
def tanh(x):
    return np.tanh(x)

## Derivative of the activation function
def tanh_prime(x):
    return 1-np.tanh(x)**2

# Loss function 
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))/2

# Derivative of the loss function

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size


class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None 

    # Add pre-defined layers to the list of layers in the network. It can add any kind of predefined layer such as FC/activation etc.
    
    def add_layer(self,layer):
        self.layers.append(layer)

    # Defining loss functions
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # Predict output for given input
    def predict(self, input_data):
        # sample dimesion first
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_prop(output)
            result.append(output)
        return result
    
    # Training the network

    def train(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)
        Error = []
        for i in range(epochs):
            err = 0
            for j in range(samples):
                if np.shape(x_train[j])[0] == 1:
                    output = x_train[j]
                else: 
                    output = x_train[j].reshape(1,len(x_train[j]))
                # Forward propagagtion 
                for layer in self.layers:
                    output = layer.forward_prop(output)

                err += self.loss(y_train[j],output)
                # Backward propagation:

                output_error = self.loss_prime(y_train[j],output)

                for layer in reversed(self.layers):
                    output_error = layer.back_prop(output_error,learning_rate)
            err /= samples
            print('epoch %d/%d   output_error=%f' % (i+1, epochs, err))


layer1 = FClayer(2,3)
# layer1.forward_prop(np.array([[1,2]]))
layer2 = FClayer(3,4)

### XOR problem

In [ ]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from scipy.stats import norm
import typing 

# training data
x_train = np.array([[[0,0,0,0]], [[0,1,0,0]], [[1,0,0,0]], [[1,1,0,0]],[[0,0,1,1]], [[0,1,1,1]], [[1,0,1,1]], [[1,1,1,1]]])
y_train = np.array([[[0]], [[0]], [[1]], [[1]],[[1]], [[1]], [[0]], [[0]]])
np.shape(y_train[0])

len(x_train[0])

In [ ]:
## Initiailzing the network

XOR_net = Network()
XOR_net.add_layer(FClayer(4, 4))
XOR_net.add_layer(Activation_layer(tanh, tanh_prime))
XOR_net.add_layer(FClayer(4, 1))
XOR_net.add_layer(Activation_layer(tanh, tanh_prime))

XOR_net.use(mse, mse_prime)
XOR_net.train(x_train, y_train, epochs=2000, learning_rate=0.1)

out = XOR_net.predict(x_train)
print(y_train.T)
print(out)

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()

digits.data = digits.data/16

x_train = digits.data[0:len(digits.data)-10]
x_test = digits.data[len(digits.data)-10:-1]
y_train = digits.target[0:len(digits.target)-10]
y_test = digits.target[len(digits.target)-10:-1]

Image_net = Network()

Image_net.add_layer(FClayer(64, 30))
Image_net.add_layer(Activation_layer(tanh, tanh_prime))
Image_net.add_layer(FClayer(30, 20))
Image_net.add_layer(Activation_layer(tanh, tanh_prime))
Image_net.add_layer(FClayer(20, 1))

Image_net.use(mse, mse_prime)
Image_net.train(x_train, y_train, epochs=500, learning_rate=0.00005)

out = Image_net.predict(x_test)
print('prediction is', out)
print()
print('Actual output is',y_test)

### Testing forward and backward propagation

In [ ]:
x_train = np.array([[[0.0,0.0]], [[0.0,1.0]], [[1.0,0.0]], [[1.0,1.0]]])
# x_train = np.random.normal(0,1,size=(4,1,2))

layer1 = FClayer(2,4)

output_l1 =layer1.forward_prop(x_train[0])

print(output_l1)
print()

Alayer1 = Activation_layer(tanh,tanh_prime)

output_Al1 = Alayer1.forward_prop(output_l1)

layer2 = FClayer(4,6)
output_l2 = layer2.forward_prop(output_Al1)

Alayer2 = Activation_layer(tanh,tanh_prime)
output_Al2 = Alayer2.forward_prop(output_l2)

layer3 = FClayer(6,2)

output_l3 = layer3.forward_prop(output_l2)

output_error_dummy = tanh_prime(output_l3) 


output_error_l3 = layer3.back_prop(output_error_dummy, 0.1)

output_error_Al2 = Alayer2.back_prop(output_error_l3)

output_error_l2 = layer2.back_prop(output_error_Al2,0.1)

# print(layer1.weights)

output_error_Al1 = Alayer1.back_prop(output_error_l2)

output_error_l1 = layer1.back_prop(output_error_Al1,1)

print()
# print(layer1.weights)
print()

output_l1_new =layer1.forward_prop(x_train[0])
print(output_l1_new)

output_error_dummy

In [ ]:
import gzip

f = gzip.open('/Users/paarthdudani/Downloads/train-images-idx3-ubyte.gz','r')

image_size = 28
num_images = 60000

f.read(16)

buf = f.read(image_size * image_size * num_images)

data_train = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
data_train = data_train.reshape(num_images, image_size, image_size, 1)

print(np.shape(data_train))

import matplotlib.pyplot as plt
image = np.asarray(data_train[0]).squeeze()
plt.imshow(image)
plt.show()

In [ ]:
f_test = gzip.open('/Users/paarthdudani/Downloads/t10k-images-idx3-ubyte.gz','r')

image_size = 28
num_images = 2000

buf_test = f_test.read(image_size * image_size * num_images)

data_test = np.frombuffer(buf_test, dtype=np.uint8).astype(np.float32)
data_test = data_test.reshape(num_images, image_size, image_size, 1)

print(np.shape(data_test))

import matplotlib.pyplot as plt
image_test = np.asarray(data_test[0]).squeeze()
plt.imshow(image_test)
plt.show()

In [ ]:
import pickle

# initializing data to be stored in db
Omkar = {'key' : 'Omkar', 'name' : 'Omkar Pathak', 
'age' : 21, 'pay' : 40000}
Jagdish = {'key' : 'Jagdish', 'name' : 'Jagdish Pathak',
'age' : 50, 'pay' : 50000}

# database
db = {}
db['Omkar'] = Omkar
db['Jagdish'] = Jagdish

# For storing
# type(b) gives <class 'bytes'>;
# b = pickle.dumps(db) 
# type(b)

# loading_pickle = pickle.loads(b)
# print(myEntry)

In [ ]:
f = gzip.open('/Users/paarthdudani/Downloads/cifar-10-python.tar.gz','r')

f.read(10*10)


image_size = 20
num_images = 1000

buf = f.read(image_size * image_size * num_images)

data_train = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
data_train = data_train.reshape(num_images, image_size, image_size, 1)

print(np.shape(data_train))


import matplotlib.pyplot as plt
image = np.asarray(data_train[0]).squeeze()
plt.imshow(image)
plt.show()

### NN trained on MNIST dataset

In [ ]:
import tensorflow as tf
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

MNIST_net = Network()

## Pre-processing to convert x_train data into 60000 samples of 1D rows.

# class data_set:
#     def __init__(self, input):
#         self.data = input
#         self.stacks = np.shape(self.data)[0]
#         self.rows = np.shape(self.data)[1]
#         self.cols = np.shape(self.data)[2]
#     def NN_friendly_reshaping(self):
#         self.data = self.data.reshape(self.stacks,1,self.rows*self.cols)
#         return self.data

x_train = x_train.astype('float32')
x_train = x_train/255
x_test = x_test.astype('float32')
x_test = x_test/255

def NN_friendly_reshaping_func(input):
    stacks = np.shape(input)[0]
    rows = np.shape(input)[1]
    cols = np.shape(input)[2]
    return input.reshape(stacks,1,rows*cols)

x_train = NN_friendly_reshaping_func(x_train)
x_test = NN_friendly_reshaping_func(x_test)


## Initializing the network

input_size = np.shape(x_train)[-1]

def k_to_1_categorizing(input):
    
    y_train_cat = np.zeros((len(input),10))
    for i in range(len(input)):
        index = input[i]
        y_train_cat[i][index] = 1
    return y_train_cat

y_train = k_to_1_categorizing(y_train)
y_test = k_to_1_categorizing(y_test)

MNIST_net.add_layer(FClayer(input_size, 80))
MNIST_net.add_layer(Activation_layer(tanh, tanh_prime))
MNIST_net.add_layer(FClayer(80, 50))
MNIST_net.add_layer(Activation_layer(tanh, tanh_prime))
MNIST_net.add_layer(FClayer(50, 10))
MNIST_net.add_layer(Activation_layer(tanh, tanh_prime))

MNIST_net.use(mse,mse_prime)
MNIST_net.train(x_train[0:1000],y_train[0:1000],epochs=40,learning_rate=0.1)

out = MNIST_net.predict(x_test[0:3])
print('prediction is', out)
print()
print('Actual output is',y_test[0:3])

### NN trained on CIFAR-10 dataset

In [ ]:
import tensorflow as tf
tf.keras.datasets.cifar10.load_data()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

sample_size = 2000

def greyscaling_func(input):
    stacks = np.shape(input)[0]
    rows = np.shape(input)[1]
    cols = np.shape(input)[2]
    greyscaled_input = np.zeros((stacks,rows,cols),)
    for i in range(stacks):
        for j in range(rows):
            for k in range(cols):
                greyscaled_input[i][j][k] = np.mean(input[i][j][k])
    
    return greyscaled_input.reshape(stacks,1,rows*cols)

x_train = greyscaling_func(x_train[0:sample_size])
x_test = greyscaling_func(x_test[0:sample_size])

x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.astype('float32')
x_test /= 255

y_train = k_to_1_categorizing(y_train[0:sample_size])
y_test = k_to_1_categorizing(y_test[0:sample_size])

input_size = np.shape(x_train)[-1]

CIFAR_10_net = Network()
CIFAR_10_net.add_layer(FClayer(input_size, 100))
CIFAR_10_net.add_layer(Activation_layer(tanh, tanh_prime))
CIFAR_10_net.add_layer(FClayer(100, 50))
CIFAR_10_net.add_layer(Activation_layer(tanh, tanh_prime))
CIFAR_10_net.add_layer(FClayer(50, 10))
CIFAR_10_net.add_layer(Activation_layer(tanh, tanh_prime))

CIFAR_10_net.use(mse,mse_prime)
CIFAR_10_net.train(x_train[0:sample_size],y_train[0:sample_size],epochs=35,learning_rate=0.05)

out = CIFAR_10_net.predict(x_test[0:3])
print('prediction is', out)
print()
print('Actual output is',y_test[0:3])